In [1]:
import requests
import pandas as pd

url_prefix = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-evaluation/'
docs_url = url_prefix + 'search_evaluation/documents-with-ids.json'
documents = requests.get(docs_url).json()

ground_truth_url = url_prefix + 'search_evaluation/ground-truth-data.csv'
df_ground_truth = pd.read_csv(ground_truth_url)
ground_truth = df_ground_truth.to_dict(orient='records')

In [2]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [3]:
ground_truth[0]

{'question': 'When does the course begin?',
 'course': 'data-engineering-zoomcamp',
 'document': 'c02e79ef'}

In [4]:
doc_idx = {d['id']: d for d in documents}

In [5]:
from tqdm.auto import tqdm

def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(query = q['question'], course = q['course'])
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Q1. Minsearch Text
## Index Data

In [6]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "section", "text"],
    keyword_fields=["course", "id"]
)

index.fit(documents)

In [7]:
boost = {'question': 1.5, 'section': 0.1}

def minsearch_search(query, course):
    results = index.search(
        query=query, 
        boost_dict=boost,
        filter_dict={'course': course},
        num_results=5
    )

    return results

In [8]:
evaluate(ground_truth, minsearch_search)

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 4627/4627 [00:13<00:00, 330.62it/s]


{'hit_rate': 0.848714069591528, 'mrr': 0.7288235717887772}

## Embeddings

In [9]:
from minsearch import VectorSearch

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

In [11]:
# Create Embeddings for the "question" field
texts = []

for doc in documents:
    t = doc['question']
    texts.append(t)

pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)
X = pipeline.fit_transform(texts)

## Q2. Vector search for question

In [12]:
vindex = VectorSearch(keyword_fields={'course'})
vindex.fit(X, documents)

In [44]:
from tqdm.auto import tqdm

def minsearch_search_fn(query, course, k=5):
    """
    Embed the query with the SAME fitted pipeline and search in Minsearch with a course filter.
    """
    # TF-IDF expects an iterable -> wrap in a list
    q_mat = pipeline.transform([query])       # shape: (1, n_components)
    # Make a 1D vector
    q_vec = q_mat.toarray()[0] if hasattr(q_mat, "toarray") else q_mat[0]

    return vindex.search(q_vec, filter_dict={'course': course}, num_results=k)

def mrr_at_k(ground_truth, k=5):
    """
    Compute Mean Reciprocal Rank@k.
    Assumes each ground_truth item has keys: 'question', 'course', 'document' (relevant doc id).
    """
    total_rr = 0.0
    n = 0

    for q in tqdm(ground_truth):
        results = minsearch_search_fn(q['question'], q['course'], k=k)
        # Find rank (1-based) of the relevant document
        rank = None
        for i, d in enumerate(results):
            if d.get('id') == q['document']:
                rank = i + 1
                break
        if rank is not None:
            total_rr += 1.0 / rank  # reciprocal rank (0 if not found)
        n += 1

    return total_rr / n if n else 0.0

# --- Run it ---
mrr5 = mrr_at_k(ground_truth, k=5)
print({"MRR@5": mrr5})


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 4627/4627 [00:06<00:00, 741.70it/s]

{'MRR@5': 0.3568510914199265}
